# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 8:30AM,241138,10,8235828,Yusen – 3D Matrix,Released
1,Jul 6 2022 8:30AM,241137,10,PRF-32748,Bio-PRF,Released
2,Jul 5 2022 3:17PM,241136,10,0085837374,ISDIN Corporation,Released
3,Jul 5 2022 3:17PM,241136,10,0085837390,ISDIN Corporation,Released
4,Jul 5 2022 3:17PM,241136,10,0085837394,ISDIN Corporation,Released
5,Jul 5 2022 3:17PM,241136,10,0085837393,ISDIN Corporation,Released
6,Jul 5 2022 3:17PM,241136,10,0085837392,ISDIN Corporation,Released
7,Jul 5 2022 3:09PM,241130,10,0085837093,ISDIN Corporation,Released
8,Jul 5 2022 3:09PM,241130,10,0085837098,ISDIN Corporation,Released
9,Jul 5 2022 3:09PM,241130,10,0085837099,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
71,241132,Released,6
72,241133,Released,7
73,241136,Released,5
74,241137,Released,1
75,241138,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241132,NaN,NaN,6.0
241133,NaN,NaN,7.0
241136,NaN,NaN,5.0
241137,NaN,NaN,1.0
241138,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241016,0.0,21.0,39.0
241017,0.0,0.0,60.0
241019,0.0,0.0,55.0
241029,8.0,3.0,17.0
241033,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241016,0,21,39
241017,0,0,60
241019,0,0,55
241029,8,3,17
241033,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241016,0,21,39
1,241017,0,0,60
2,241019,0,0,55
3,241029,8,3,17
4,241033,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241016,,21,39
1,241017,,,60
2,241019,,,55
3,241029,8,3,17
4,241033,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 8:30AM,241138,10,Yusen – 3D Matrix
1,Jul 6 2022 8:30AM,241137,10,Bio-PRF
2,Jul 5 2022 3:17PM,241136,10,ISDIN Corporation
7,Jul 5 2022 3:09PM,241130,10,ISDIN Corporation
18,Jul 5 2022 3:08PM,241129,10,ISDIN Corporation
70,Jul 5 2022 3:08PM,241131,10,ISDIN Corporation
72,Jul 5 2022 3:08PM,241128,10,ISDIN Corporation
105,Jul 5 2022 3:08PM,241127,10,ISDIN Corporation
137,Jul 5 2022 3:08PM,241126,10,ISDIN Corporation
148,Jul 5 2022 3:05PM,241132,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 8:30AM,241138,10,Yusen – 3D Matrix,,,1
1,Jul 6 2022 8:30AM,241137,10,Bio-PRF,,,1
2,Jul 5 2022 3:17PM,241136,10,ISDIN Corporation,,,5
3,Jul 5 2022 3:09PM,241130,10,ISDIN Corporation,,,11
4,Jul 5 2022 3:08PM,241129,10,ISDIN Corporation,,,52
5,Jul 5 2022 3:08PM,241131,10,ISDIN Corporation,,,2
6,Jul 5 2022 3:08PM,241128,10,ISDIN Corporation,,,33
7,Jul 5 2022 3:08PM,241127,10,ISDIN Corporation,,,32
8,Jul 5 2022 3:08PM,241126,10,ISDIN Corporation,,,11
9,Jul 5 2022 3:05PM,241132,10,"Methapharm, Inc.",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 8:30AM,241138,10,Yusen – 3D Matrix,1,,
1,Jul 6 2022 8:30AM,241137,10,Bio-PRF,1,,
2,Jul 5 2022 3:17PM,241136,10,ISDIN Corporation,5,,
3,Jul 5 2022 3:09PM,241130,10,ISDIN Corporation,11,,
4,Jul 5 2022 3:08PM,241129,10,ISDIN Corporation,52,,
5,Jul 5 2022 3:08PM,241131,10,ISDIN Corporation,2,,
6,Jul 5 2022 3:08PM,241128,10,ISDIN Corporation,33,,
7,Jul 5 2022 3:08PM,241127,10,ISDIN Corporation,32,,
8,Jul 5 2022 3:08PM,241126,10,ISDIN Corporation,11,,
9,Jul 5 2022 3:05PM,241132,10,"Methapharm, Inc.",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 8:30AM,241138,10,Yusen – 3D Matrix,1,,
1,Jul 6 2022 8:30AM,241137,10,Bio-PRF,1,,
2,Jul 5 2022 3:17PM,241136,10,ISDIN Corporation,5,,
3,Jul 5 2022 3:09PM,241130,10,ISDIN Corporation,11,,
4,Jul 5 2022 3:08PM,241129,10,ISDIN Corporation,52,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 8:30AM,241138,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Jul 6 2022 8:30AM,241137,10,Bio-PRF,1.0,NaN,NaN
2,Jul 5 2022 3:17PM,241136,10,ISDIN Corporation,5.0,NaN,NaN
3,Jul 5 2022 3:09PM,241130,10,ISDIN Corporation,11.0,NaN,NaN
4,Jul 5 2022 3:08PM,241129,10,ISDIN Corporation,52.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 8:30AM,241138,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Jul 6 2022 8:30AM,241137,10,Bio-PRF,1.0,0.0,0.0
2,Jul 5 2022 3:17PM,241136,10,ISDIN Corporation,5.0,0.0,0.0
3,Jul 5 2022 3:09PM,241130,10,ISDIN Corporation,11.0,0.0,0.0
4,Jul 5 2022 3:08PM,241129,10,ISDIN Corporation,52.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,11330800,1384.0,36.0,8.0
12,2169787,27.0,66.0,0.0
15,241073,10.0,9.0,0.0
16,482219,2.0,0.0,0.0
19,482236,30.0,5.0,0.0
20,1205453,101.0,27.0,12.0
22,241122,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,11330800,1384.0,36.0,8.0
1,12,2169787,27.0,66.0,0.0
2,15,241073,10.0,9.0,0.0
3,16,482219,2.0,0.0,0.0
4,19,482236,30.0,5.0,0.0
5,20,1205453,101.0,27.0,12.0
6,22,241122,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1384.0,36.0,8.0
1,12,27.0,66.0,0.0
2,15,10.0,9.0,0.0
3,16,2.0,0.0,0.0
4,19,30.0,5.0,0.0
5,20,101.0,27.0,12.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1384.0
1,12,Released,27.0
2,15,Released,10.0
3,16,Released,2.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,8.0,0.0,0.0,0.0,0.0,12.0,0.0
Executing,36.0,66.0,9.0,0.0,5.0,27.0,0.0
Released,1384.0,27.0,10.0,2.0,30.0,101.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,8.0,0.0,0.0,0.0,0.0,12.0,0.0
1,Executing,36.0,66.0,9.0,0.0,5.0,27.0,0.0
2,Released,1384.0,27.0,10.0,2.0,30.0,101.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,8.0,0.0,0.0,0.0,0.0,12.0,0.0
1,Executing,36.0,66.0,9.0,0.0,5.0,27.0,0.0
2,Released,1384.0,27.0,10.0,2.0,30.0,101.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()